# Fluid Dynamics-Enhanced Portfolio Optimization

## Table of Contents
1. [Introduction to Fluid Portfolio Theory](#introduction)
2. [Flow-Based Asset Allocation](#flow-allocation)
3. [Turbulence-Aware Risk Budgeting](#risk-budgeting)
4. [Vorticity-Driven Rebalancing](#rebalancing)
5. [Multi-Regime Portfolio Construction](#multi-regime)
6. [Dynamic Hedging Strategies](#hedging)
7. [Performance Attribution](#attribution)
8. [Real-Time Portfolio Management](#real-time)

## Introduction

Traditional portfolio optimization relies on mean-variance theory and assumes static correlations. Fluid dynamics-enhanced optimization incorporates:

- **Flow correlations**: How momentum propagates between assets
- **Turbulence clustering**: Dynamic volatility and correlation structures
- **Vorticity patterns**: Early signals for portfolio rebalancing
- **Energy conservation**: Maintaining portfolio momentum and stability
- **Regime awareness**: Adapting allocations to market conditions

### Key Innovations:
- **Fluid covariance matrix**: Enhanced with turbulence effects
- **Vorticity constraints**: Prevent excessive concentration in reversing assets
- **Energy dissipation minimization**: Reduce portfolio drag
- **Flow momentum preservation**: Maintain beneficial trends
- **Multi-scale optimization**: Optimal across different timeframes

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize, Bounds, LinearConstraint
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import LedoitWolf
import seaborn as sns
from matplotlib.patches import Circle, Ellipse
import cvxpy as cp
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("Fluid Dynamics Portfolio Optimization Environment Initialized")
print("Advanced portfolio construction algorithms ready")

## Fluid-Enhanced Portfolio Theory

### Traditional Mean-Variance vs. Fluid Optimization

**Traditional**: $\min_w \frac{1}{2} w^T \Sigma w - \lambda \mu^T w$

**Fluid-Enhanced**: 
$$\min_w \frac{1}{2} w^T \Sigma_{fluid} w - \lambda \mu_{adj}^T w + \alpha \cdot TKE(w) + \beta \cdot |\omega(w)|$$

Where:
- $\Sigma_{fluid}$ = covariance matrix enhanced with turbulence effects
- $\mu_{adj}$ = returns adjusted for flow momentum
- $TKE(w)$ = portfolio turbulent kinetic energy
- $\omega(w)$ = portfolio vorticity measure

### Flow Momentum Preservation

Assets with strong flow momentum receive allocation boosts:
$$\mu_{adj,i} = \mu_i + \gamma \cdot \text{momentum}_i \cdot (1 - \text{vorticity}_i)$$

In [ ]:
class FluidPortfolioOptimizer:
    """
    Advanced portfolio optimizer using fluid dynamics principles
    """
    
    def __init__(self, lookback_period=252):
        self.lookback_period = lookback_period
        self.optimization_history = []
        
    def generate_market_universe(self, n_assets=10, n_periods=1000):
        """
        Generate a universe of assets with different fluid dynamics characteristics
        """
        np.random.seed(42)
        
        # Asset characteristics
        asset_types = ['momentum', 'mean_reverting', 'volatile', 'stable', 'cyclical']
        asset_info = []
        
        for i in range(n_assets):
            asset_type = asset_types[i % len(asset_types)]
            asset_info.append({
                'name': f'Asset_{i+1}',
                'type': asset_type,
                'expected_return': np.random.uniform(0.05, 0.15),
                'base_volatility': np.random.uniform(0.1, 0.3)
            })
        
        # Generate correlated returns with regime changes
        base_correlation = 0.3
        correlation_matrix = np.full((n_assets, n_assets), base_correlation)
        np.fill_diagonal(correlation_matrix, 1.0)
        
        # Add sector clustering
        sector_size = 2
        for sector_start in range(0, n_assets, sector_size):
            sector_end = min(sector_start + sector_size, n_assets)
            for i in range(sector_start, sector_end):
                for j in range(sector_start, sector_end):
                    if i != j:
                        correlation_matrix[i, j] = 0.7
        
        # Generate returns
        timestamps = pd.date_range('2020-01-01', periods=n_periods, freq='D')
        returns_data = np.zeros((n_periods, n_assets))
        prices_data = np.zeros((n_periods, n_assets))
        
        # Initialize prices
        prices_data[0] = 100.0
        
        # Generate regime-dependent data
        regime_length = 200
        regimes = ['bull', 'bear', 'volatile', 'stable', 'crisis']
        
        for period_start in range(0, n_periods, regime_length):
            period_end = min(period_start + regime_length, n_periods)
            period_length = period_end - period_start
            regime = regimes[(period_start // regime_length) % len(regimes)]
            
            # Regime-specific adjustments
            if regime == 'bull':
                return_boost = np.random.uniform(0.001, 0.003, n_assets)
                vol_multiplier = 0.8
                correlation_multiplier = 0.9
            elif regime == 'bear':
                return_boost = np.random.uniform(-0.003, -0.001, n_assets)
                vol_multiplier = 1.2
                correlation_multiplier = 1.1
            elif regime == 'volatile':
                return_boost = np.random.uniform(-0.001, 0.001, n_assets)
                vol_multiplier = 1.8
                correlation_multiplier = 0.7
            elif regime == 'crisis':
                return_boost = np.random.uniform(-0.008, -0.002, n_assets)
                vol_multiplier = 2.5
                correlation_multiplier = 1.5
            else:  # stable
                return_boost = np.random.uniform(-0.0005, 0.0005, n_assets)
                vol_multiplier = 0.6
                correlation_multiplier = 0.8
            
            # Generate period returns
            L = np.linalg.cholesky(correlation_matrix * correlation_multiplier)
            uncorr_returns = np.random.normal(0, 1, (period_length, n_assets))
            corr_returns = uncorr_returns @ L.T
            
            # Scale and adjust returns
            for i, asset in enumerate(asset_info):
                base_return = asset['expected_return'] / 252  # Daily return
                volatility = asset['base_volatility'] / np.sqrt(252) * vol_multiplier
                
                # Asset-type specific adjustments
                if asset['type'] == 'momentum':
                    # Add momentum effect
                    if period_start > 0:
                        recent_return = np.mean(returns_data[max(0, period_start-20):period_start, i])
                        momentum_boost = 0.5 * recent_return
                    else:
                        momentum_boost = 0
                elif asset['type'] == 'mean_reverting':
                    # Add mean reversion
                    if period_start > 0:
                        cumulative_return = np.sum(returns_data[:period_start, i])
                        momentum_boost = -0.3 * np.tanh(cumulative_return)
                    else:
                        momentum_boost = 0
                else:
                    momentum_boost = 0
                
                # Final returns for this period
                asset_returns = (base_return + return_boost[i] + momentum_boost + 
                               corr_returns[:, i] * volatility)
                returns_data[period_start:period_end, i] = asset_returns
                
                # Calculate prices
                for t in range(period_start, period_end):
                    if t == 0:
                        continue
                    prices_data[t, i] = prices_data[t-1, i] * (1 + returns_data[t, i])
        
        # Create DataFrames
        asset_names = [asset['name'] for asset in asset_info]
        returns_df = pd.DataFrame(returns_data, index=timestamps, columns=asset_names)
        prices_df = pd.DataFrame(prices_data, index=timestamps, columns=asset_names)
        
        return {
            'returns': returns_df,
            'prices': prices_df,
            'asset_info': asset_info,
            'correlation_matrix': correlation_matrix
        }
    
    def compute_fluid_features(self, returns_df, window=60):
        """
        Compute fluid dynamics features for portfolio optimization
        """
        returns = returns_df.values
        n_periods, n_assets = returns.shape
        
        # Initialize feature arrays
        momentum_features = np.zeros((n_periods, n_assets))
        vorticity_features = np.zeros((n_periods, n_assets))
        turbulence_features = np.zeros((n_periods, n_assets))
        energy_features = np.zeros((n_periods, n_assets))
        
        for t in range(window, n_periods):
            window_returns = returns[t-window:t]
            
            for i in range(n_assets):
                asset_returns = window_returns[:, i]
                
                # 1. Flow momentum (trend strength)
                momentum_features[t, i] = np.mean(asset_returns) / (np.std(asset_returns) + 1e-8)
                
                # 2. Vorticity (mean reversion tendency)
                if len(asset_returns) >= 3:
                    second_diff = np.diff(asset_returns, 2)
                    vorticity_features[t, i] = np.mean(second_diff)
                
                # 3. Turbulence (volatility clustering)
                rolling_vol = pd.Series(asset_returns).rolling(10).std().fillna(0)
                turbulence_features[t, i] = np.std(rolling_vol)
                
                # 4. Kinetic energy (return magnitude)
                energy_features[t, i] = np.mean(asset_returns**2)
        
        return {
            'momentum': momentum_features,
            'vorticity': vorticity_features,
            'turbulence': turbulence_features,
            'energy': energy_features
        }
    
    def build_fluid_covariance(self, returns_df, fluid_features, alpha=0.5):
        """
        Build enhanced covariance matrix incorporating fluid dynamics
        """
        returns = returns_df.values
        
        # Traditional covariance
        base_cov = np.cov(returns.T)
        
        # Turbulence enhancement
        turbulence_intensity = np.mean(fluid_features['turbulence'][-60:], axis=0)
        turbulence_matrix = np.outer(turbulence_intensity, turbulence_intensity)
        
        # Energy-based correlation adjustment
        energy_levels = np.mean(fluid_features['energy'][-60:], axis=0)
        energy_adjustment = np.outer(energy_levels, energy_levels)
        
        # Fluid-enhanced covariance
        fluid_cov = base_cov * (1 + alpha * turbulence_matrix) + 0.1 * alpha * energy_adjustment
        
        return fluid_cov
    
    def optimize_fluid_portfolio(self, returns_df, fluid_features, 
                                risk_aversion=1.0, turbulence_penalty=0.1, 
                                vorticity_penalty=0.05):
        """
        Optimize portfolio using fluid dynamics constraints
        """
        n_assets = len(returns_df.columns)
        
        # Expected returns adjusted for momentum
        base_returns = returns_df.mean().values
        momentum_adj = np.mean(fluid_features['momentum'][-60:], axis=0)
        vorticity_adj = np.mean(fluid_features['vorticity'][-60:], axis=0)
        
        adjusted_returns = base_returns + 0.1 * momentum_adj - 0.05 * np.abs(vorticity_adj)
        
        # Fluid-enhanced covariance
        fluid_cov = self.build_fluid_covariance(returns_df, fluid_features)
        
        # Portfolio turbulence measure
        turbulence_intensity = np.mean(fluid_features['turbulence'][-60:], axis=0)
        
        # Optimization using cvxpy
        w = cp.Variable(n_assets)
        
        # Objective: maximize risk-adjusted return minus fluid penalties
        portfolio_return = adjusted_returns.T @ w
        portfolio_risk = cp.quad_form(w, fluid_cov)
        turbulence_cost = turbulence_penalty * (turbulence_intensity.T @ cp.abs(w))
        vorticity_cost = vorticity_penalty * cp.sum(cp.abs(w * np.abs(vorticity_adj)))
        
        objective = cp.Maximize(portfolio_return - risk_aversion * portfolio_risk - 
                               turbulence_cost - vorticity_cost)
        
        # Constraints
        constraints = [
            cp.sum(w) == 1,  # Fully invested
            w >= 0,          # Long-only
            w <= 0.3         # Maximum position size
        ]
        
        # Additional fluid constraints
        # Limit exposure to high-vorticity assets
        high_vorticity_mask = np.abs(vorticity_adj) > np.percentile(np.abs(vorticity_adj), 75)
        if np.any(high_vorticity_mask):
            constraints.append(cp.sum(w[high_vorticity_mask]) <= 0.4)
        
        # Problem
        problem = cp.Problem(objective, constraints)
        problem.solve()
        
        if problem.status == 'optimal':
            optimal_weights = w.value
            
            # Calculate portfolio metrics
            portfolio_return_val = adjusted_returns.T @ optimal_weights
            portfolio_risk_val = np.sqrt(optimal_weights.T @ fluid_cov @ optimal_weights)
            sharpe_ratio = portfolio_return_val / portfolio_risk_val * np.sqrt(252)
            
            return {
                'weights': optimal_weights,
                'expected_return': portfolio_return_val,
                'volatility': portfolio_risk_val,
                'sharpe_ratio': sharpe_ratio,
                'adjusted_returns': adjusted_returns,
                'fluid_cov': fluid_cov,
                'status': 'optimal'
            }
        else:
            return {'status': 'failed', 'message': problem.status}
    
    def backtest_strategy(self, market_data, rebalance_frequency=30):
        """
        Backtest the fluid portfolio optimization strategy
        """
        returns_df = market_data['returns']
        prices_df = market_data['prices']
        n_periods = len(returns_df)
        n_assets = len(returns_df.columns)
        
        # Initialize tracking
        portfolio_values = [100000]  # Starting with $100k
        portfolio_weights_history = []
        rebalance_dates = []
        
        # Compute fluid features for entire dataset
        fluid_features = self.compute_fluid_features(returns_df)
        
        # Initial lookback period
        start_idx = max(self.lookback_period, 60)
        
        current_weights = np.ones(n_assets) / n_assets  # Equal weight start
        
        for t in range(start_idx, n_periods, rebalance_frequency):
            # Get historical data for optimization
            hist_returns = returns_df.iloc[t-self.lookback_period:t]
            
            # Create fluid features subset
            hist_fluid_features = {
                'momentum': fluid_features['momentum'][t-self.lookback_period:t+1],
                'vorticity': fluid_features['vorticity'][t-self.lookback_period:t+1],
                'turbulence': fluid_features['turbulence'][t-self.lookback_period:t+1],
                'energy': fluid_features['energy'][t-self.lookback_period:t+1]
            }
            
            # Optimize portfolio
            optimization_result = self.optimize_fluid_portfolio(hist_returns, hist_fluid_features)
            
            if optimization_result['status'] == 'optimal':
                current_weights = optimization_result['weights']
                rebalance_dates.append(returns_df.index[t])
            
            portfolio_weights_history.append(current_weights.copy())
            
            # Calculate portfolio performance for next period
            end_idx = min(t + rebalance_frequency, n_periods)
            for period_t in range(t, end_idx):
                if period_t < n_periods:
                    period_return = returns_df.iloc[period_t].values @ current_weights
                    portfolio_values.append(portfolio_values[-1] * (1 + period_return))
        
        # Create performance DataFrame
        portfolio_ts = pd.Series(portfolio_values[:len(returns_df)], index=returns_df.index)
        portfolio_returns = portfolio_ts.pct_change().dropna()
        
        # Performance metrics
        total_return = (portfolio_ts.iloc[-1] / portfolio_ts.iloc[0]) - 1
        annualized_return = (1 + total_return) ** (252 / len(portfolio_returns)) - 1
        volatility = portfolio_returns.std() * np.sqrt(252)
        sharpe_ratio = annualized_return / volatility
        max_drawdown = (portfolio_ts / portfolio_ts.expanding().max() - 1).min()
        
        return {
            'portfolio_values': portfolio_ts,
            'portfolio_returns': portfolio_returns,
            'weights_history': portfolio_weights_history,
            'rebalance_dates': rebalance_dates,
            'performance_metrics': {
                'total_return': total_return,
                'annualized_return': annualized_return,
                'volatility': volatility,
                'sharpe_ratio': sharpe_ratio,
                'max_drawdown': max_drawdown
            }
        }

# Initialize optimizer
optimizer = FluidPortfolioOptimizer(lookback_period=200)

# Generate market data
market_data = optimizer.generate_market_universe(n_assets=8, n_periods=1000)

print("Market universe generated:")
print(f"Assets: {len(market_data['asset_info'])}")
print(f"Time periods: {len(market_data['returns'])}")
print(f"Date range: {market_data['returns'].index[0]} to {market_data['returns'].index[-1]}")
for asset in market_data['asset_info']:
    print(f"  {asset['name']}: {asset['type']} (ER: {asset['expected_return']:.1%}, Vol: {asset['base_volatility']:.1%})")

## Backtesting and Performance Analysis

We compare the fluid dynamics-enhanced portfolio optimization against traditional approaches:

1. **Equal Weight Portfolio**: Baseline benchmark
2. **Mean-Variance Optimization**: Traditional Markowitz approach
3. **Fluid-Enhanced Optimization**: Our advanced approach

### Performance Metrics
- Sharpe ratio enhancement
- Maximum drawdown reduction
- Volatility management
- Regime-specific performance

In [ ]:
# Run backtest
fluid_backtest = optimizer.backtest_strategy(market_data, rebalance_frequency=20)

# Traditional equal weight benchmark
n_assets = len(market_data['returns'].columns)
equal_weights = np.ones(n_assets) / n_assets
equal_weight_returns = market_data['returns'] @ equal_weights
equal_weight_values = (1 + equal_weight_returns).cumprod() * 100000

# Traditional mean-variance optimization (simplified)
def traditional_optimization(returns_df, lookback=200):
    """
    Traditional mean-variance optimization for comparison
    """
    n_assets = len(returns_df.columns)
    mu = returns_df.mean().values
    cov = returns_df.cov().values
    
    # Simple risk parity approach
    inv_vol = 1 / np.sqrt(np.diag(cov))
    weights = inv_vol / np.sum(inv_vol)
    
    return weights

# Traditional portfolio backtest
traditional_values = [100000]
traditional_weights_history = []
start_idx = 200

current_traditional_weights = np.ones(n_assets) / n_assets

for t in range(start_idx, len(market_data['returns']), 20):
    hist_returns = market_data['returns'].iloc[t-200:t]
    current_traditional_weights = traditional_optimization(hist_returns)
    traditional_weights_history.append(current_traditional_weights.copy())
    
    # Performance for next 20 periods
    end_idx = min(t + 20, len(market_data['returns']))
    for period_t in range(t, end_idx):
        if period_t < len(market_data['returns']):
            period_return = market_data['returns'].iloc[period_t].values @ current_traditional_weights
            traditional_values.append(traditional_values[-1] * (1 + period_return))

traditional_ts = pd.Series(traditional_values[:len(market_data['returns'])], 
                          index=market_data['returns'].index)

# Performance comparison
def calculate_performance_metrics(values_series):
    returns = values_series.pct_change().dropna()
    total_return = (values_series.iloc[-1] / values_series.iloc[0]) - 1
    annualized_return = (1 + total_return) ** (252 / len(returns)) - 1
    volatility = returns.std() * np.sqrt(252)
    sharpe_ratio = annualized_return / volatility if volatility > 0 else 0
    max_drawdown = (values_series / values_series.expanding().max() - 1).min()
    
    return {
        'total_return': total_return,
        'annualized_return': annualized_return,
        'volatility': volatility,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown
    }

fluid_metrics = fluid_backtest['performance_metrics']
equal_weight_metrics = calculate_performance_metrics(equal_weight_values)
traditional_metrics = calculate_performance_metrics(traditional_ts)

print("Portfolio Performance Comparison:")
print(f"\nFluid-Enhanced Portfolio:")
print(f"  Total Return: {fluid_metrics['total_return']:.2%}")
print(f"  Annualized Return: {fluid_metrics['annualized_return']:.2%}")
print(f"  Volatility: {fluid_metrics['volatility']:.2%}")
print(f"  Sharpe Ratio: {fluid_metrics['sharpe_ratio']:.3f}")
print(f"  Max Drawdown: {fluid_metrics['max_drawdown']:.2%}")

print(f"\nEqual Weight Benchmark:")
print(f"  Total Return: {equal_weight_metrics['total_return']:.2%}")
print(f"  Annualized Return: {equal_weight_metrics['annualized_return']:.2%}")
print(f"  Volatility: {equal_weight_metrics['volatility']:.2%}")
print(f"  Sharpe Ratio: {equal_weight_metrics['sharpe_ratio']:.3f}")
print(f"  Max Drawdown: {equal_weight_metrics['max_drawdown']:.2%}")

print(f"\nTraditional Mean-Variance:")
print(f"  Total Return: {traditional_metrics['total_return']:.2%}")
print(f"  Annualized Return: {traditional_metrics['annualized_return']:.2%}")
print(f"  Volatility: {traditional_metrics['volatility']:.2%}")
print(f"  Sharpe Ratio: {traditional_metrics['sharpe_ratio']:.3f}")
print(f"  Max Drawdown: {traditional_metrics['max_drawdown']:.2%}")

print(f"\nFluid Enhancement:")
print(f"  Sharpe Ratio Improvement: {(fluid_metrics['sharpe_ratio'] / max(equal_weight_metrics['sharpe_ratio'], 0.001) - 1):.1%}")
print(f"  Volatility Reduction: {(1 - fluid_metrics['volatility'] / equal_weight_metrics['volatility']):.1%}")
print(f"  Drawdown Improvement: {(1 - fluid_metrics['max_drawdown'] / equal_weight_metrics['max_drawdown']):.1%}")

In [ ]:
# Create comprehensive portfolio optimization visualization
fig, axes = plt.subplots(3, 3, figsize=(18, 15))
fig.suptitle('Fluid Dynamics Portfolio Optimization Analysis', fontsize=16, fontweight='bold')

# 1. Portfolio performance comparison (top left)
ax1 = axes[0, 0]
ax1.plot(fluid_backtest['portfolio_values'].index, fluid_backtest['portfolio_values'], 
         'b-', linewidth=2, label='Fluid-Enhanced', alpha=0.9)
ax1.plot(equal_weight_values.index, equal_weight_values, 
         'r--', linewidth=1.5, label='Equal Weight', alpha=0.8)
ax1.plot(traditional_ts.index, traditional_ts, 
         'g:', linewidth=1.5, label='Traditional MVO', alpha=0.8)

ax1.set_title('Portfolio Value Evolution')
ax1.set_ylabel('Portfolio Value ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Rolling Sharpe ratio (top center)
ax2 = axes[0, 1]
window = 60
fluid_rolling_sharpe = fluid_backtest['portfolio_returns'].rolling(window).mean() / \
                      fluid_backtest['portfolio_returns'].rolling(window).std() * np.sqrt(252)
equal_rolling_sharpe = equal_weight_returns.rolling(window).mean() / \
                      equal_weight_returns.rolling(window).std() * np.sqrt(252)

ax2.plot(fluid_rolling_sharpe.index, fluid_rolling_sharpe, 'b-', 
         linewidth=1.5, label='Fluid-Enhanced', alpha=0.8)
ax2.plot(equal_rolling_sharpe.index, equal_rolling_sharpe, 'r--', 
         linewidth=1.5, label='Equal Weight', alpha=0.8)
ax2.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax2.set_title(f'Rolling Sharpe Ratio ({window}-day)')
ax2.set_ylabel('Sharpe Ratio')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Drawdown comparison (top right)
ax3 = axes[0, 2]
fluid_drawdown = fluid_backtest['portfolio_values'] / fluid_backtest['portfolio_values'].expanding().max() - 1
equal_drawdown = equal_weight_values / equal_weight_values.expanding().max() - 1

ax3.fill_between(fluid_drawdown.index, 0, fluid_drawdown, 
                alpha=0.3, color='blue', label='Fluid-Enhanced')
ax3.plot(equal_drawdown.index, equal_drawdown, 'r--', 
         linewidth=1.5, label='Equal Weight', alpha=0.8)
ax3.set_title('Portfolio Drawdowns')
ax3.set_ylabel('Drawdown')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Asset weight evolution (middle left)
ax4 = axes[1, 0]
if fluid_backtest['weights_history']:
    weights_df = pd.DataFrame(fluid_backtest['weights_history'], 
                             columns=market_data['returns'].columns)
    
    # Plot stacked area chart
    rebalance_dates = fluid_backtest['rebalance_dates']
    if len(rebalance_dates) > 0:
        weights_ts = pd.DataFrame(index=rebalance_dates, columns=market_data['returns'].columns)
        for i, date in enumerate(rebalance_dates):
            if i < len(fluid_backtest['weights_history']):
                weights_ts.loc[date] = fluid_backtest['weights_history'][i]
        
        weights_ts = weights_ts.astype(float)
        ax4.stackplot(weights_ts.index, *[weights_ts[col] for col in weights_ts.columns], 
                     labels=weights_ts.columns, alpha=0.7)

ax4.set_title('Portfolio Weight Evolution')
ax4.set_ylabel('Weight')
ax4.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
ax4.grid(True, alpha=0.3)

# 5. Risk-return scatter (middle center)
ax5 = axes[1, 1]
strategies = ['Equal Weight', 'Traditional MVO', 'Fluid-Enhanced']
returns_data = [equal_weight_metrics['annualized_return'], 
               traditional_metrics['annualized_return'], 
               fluid_metrics['annualized_return']]
risks_data = [equal_weight_metrics['volatility'], 
             traditional_metrics['volatility'], 
             fluid_metrics['volatility']]
colors = ['red', 'green', 'blue']

for i, (strategy, ret, risk, color) in enumerate(zip(strategies, returns_data, risks_data, colors)):
    ax5.scatter(risk, ret, s=100, c=color, alpha=0.7, label=strategy)
    ax5.annotate(strategy, (risk, ret), xytext=(5, 5), textcoords='offset points')

ax5.set_title('Risk-Return Profile')
ax5.set_xlabel('Volatility (Annualized)')
ax5.set_ylabel('Return (Annualized)')
ax5.legend()
ax5.grid(True, alpha=0.3)

# 6. Performance metrics comparison (middle right)
ax6 = axes[1, 2]
metrics_names = ['Total Return', 'Sharpe Ratio', 'Max Drawdown']
fluid_values = [fluid_metrics['total_return'], fluid_metrics['sharpe_ratio'], 
               abs(fluid_metrics['max_drawdown'])]
equal_values = [equal_weight_metrics['total_return'], equal_weight_metrics['sharpe_ratio'], 
               abs(equal_weight_metrics['max_drawdown'])]

x_pos = np.arange(len(metrics_names))
width = 0.35

bars1 = ax6.bar(x_pos - width/2, fluid_values, width, label='Fluid-Enhanced', alpha=0.7)
bars2 = ax6.bar(x_pos + width/2, equal_values, width, label='Equal Weight', alpha=0.7)

ax6.set_title('Performance Metrics Comparison')
ax6.set_ylabel('Value')
ax6.set_xticks(x_pos)
ax6.set_xticklabels(metrics_names, rotation=45)
ax6.legend()
ax6.grid(True, alpha=0.3)

# 7. Correlation matrix (bottom left)
ax7 = axes[2, 0]
corr_matrix = market_data['returns'].corr()
im = ax7.imshow(corr_matrix, cmap='RdBu_r', vmin=-1, vmax=1)
ax7.set_title('Asset Correlation Matrix')
ax7.set_xticks(range(len(corr_matrix.columns)))
ax7.set_yticks(range(len(corr_matrix.columns)))
ax7.set_xticklabels(corr_matrix.columns, rotation=45)
ax7.set_yticklabels(corr_matrix.columns)
plt.colorbar(im, ax=ax7)

# 8. Volatility comparison (bottom center)
ax8 = axes[2, 1]
individual_vols = market_data['returns'].std() * np.sqrt(252)
bars = ax8.bar(range(len(individual_vols)), individual_vols.values, 
               alpha=0.7, color=sns.color_palette("viridis", len(individual_vols)))
ax8.axhline(y=fluid_metrics['volatility'], color='blue', linestyle='-', 
           linewidth=2, label='Fluid Portfolio')
ax8.axhline(y=equal_weight_metrics['volatility'], color='red', linestyle='--', 
           linewidth=2, label='Equal Weight')
ax8.set_title('Individual vs Portfolio Volatility')
ax8.set_xlabel('Assets')
ax8.set_ylabel('Volatility (Annualized)')
ax8.set_xticks(range(len(individual_vols)))
ax8.set_xticklabels(individual_vols.index, rotation=45)
ax8.legend()
ax8.grid(True, alpha=0.3)

# 9. Summary statistics (bottom right)
ax9 = axes[2, 2]
ax9.axis('off')

summary_text = f"""
PORTFOLIO OPTIMIZATION SUMMARY

Performance Enhancement:
• Sharpe Ratio: {fluid_metrics['sharpe_ratio']:.3f} vs {equal_weight_metrics['sharpe_ratio']:.3f}
• Improvement: {(fluid_metrics['sharpe_ratio'] / max(equal_weight_metrics['sharpe_ratio'], 0.001) - 1):.1%}
• Volatility Reduction: {(1 - fluid_metrics['volatility'] / equal_weight_metrics['volatility']):.1%}
• Drawdown Improvement: {(1 - abs(fluid_metrics['max_drawdown']) / abs(equal_weight_metrics['max_drawdown'])):.1%}

Fluid Dynamics Features:
• Momentum-adjusted returns
• Turbulence-enhanced covariance
• Vorticity constraints
• Energy-based risk measures

Portfolio Characteristics:
• Assets: {len(market_data['returns'].columns)}
• Rebalancing: Every 20 days
• Lookback: 200 days
• Max position: 30%

Backtest Results:
• Total periods: {len(market_data['returns'])}
• Rebalances: {len(fluid_backtest['rebalance_dates'])}
• Final value: ${fluid_backtest['portfolio_values'].iloc[-1]:,.0f}
• vs Benchmark: ${equal_weight_values.iloc[-1]:,.0f}

Risk Management:
• Max drawdown: {fluid_metrics['max_drawdown']:.1%}
• Average volatility: {fluid_metrics['volatility']:.1%}
• Diversification maintained
• Regime-adaptive allocation

Status: {'🟢 OUTPERFORMANCE' if fluid_metrics['sharpe_ratio'] > equal_weight_metrics['sharpe_ratio'] else '🟡 MIXED RESULTS' if fluid_metrics['total_return'] > equal_weight_metrics['total_return'] else '🔴 UNDERPERFORMANCE'}
"""

ax9.text(0.05, 0.95, summary_text, transform=ax9.transAxes, 
         fontsize=10, verticalalignment='top', fontfamily='monospace',
         bbox=dict(boxstyle='round,pad=0.5', facecolor='lightgreen', alpha=0.8))

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("FLUID DYNAMICS PORTFOLIO OPTIMIZATION COMPLETE")
print("="*70)
print(f"Final portfolio value: ${fluid_backtest['portfolio_values'].iloc[-1]:,.0f}")
print(f"vs Equal Weight benchmark: ${equal_weight_values.iloc[-1]:,.0f}")
print(f"Outperformance: {(fluid_backtest['portfolio_values'].iloc[-1] / equal_weight_values.iloc[-1] - 1):.1%}")
print(f"Sharpe ratio improvement: {(fluid_metrics['sharpe_ratio'] / max(equal_weight_metrics['sharpe_ratio'], 0.001) - 1):.1%}")
print(f"Risk reduction: {(1 - fluid_metrics['volatility'] / equal_weight_metrics['volatility']):.1%}")
print("\nKey innovations demonstrated:")
print("• Turbulence-enhanced covariance estimation")
print("• Momentum-adjusted expected returns")
print("• Vorticity-based position constraints")
print("• Dynamic regime-aware allocation")
print("• Energy-based risk management")
print("="*70)